In [1]:
# -*- coding:utf-8
import sys
import os
import importlib
importlib.reload(sys)
from pdfminer.pdfparser import PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
import jieba
 
'''
 解析pdf 文本，保存到txt文件中
'''
root_path = R'C:\Users\zgj_t\Desktop\SummerCourse'

def parse_pdf(name):
    pdf_name = name+'.pdf'
    txt_name = name+'.txt'
    pdf_path = os.path.join(root_path, pdf_name)
    txt_path = os.path.join(root_path, txt_name)
    fp = open(pdf_path, 'rb') # 以二进制读模式打开
    #用文件对象来创建一个pdf文档分析器
    praser = PDFParser(fp)
    # 创建一个PDF文档
    doc = PDFDocument()
    # 连接分析器 与文档对象
    praser.set_document(doc)
    doc.set_parser(praser)
 
    # 提供初始化密码
    # 如果没有密码 就创建一个空的字符串
    doc.initialize()
 
    # 检测文档是否提供txt转换，不提供就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        # 创建PDf 资源管理器 来管理共享资源
        rsrcmgr = PDFResourceManager()
        # 创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        # 创建一个PDF解释器对象
        interpreter = PDFPageInterpreter(rsrcmgr, device)
 
        # 循环遍历列表，每次处理一个page的内容
        for page in doc.get_pages(): # doc.get_pages() 获取page列表
            interpreter.process_page(page)
            # 接受该页面的LTPage对象
            layout = device.get_result()
            # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等 想要获取文本就获得对象的text属性，
            for x in layout:
                if (isinstance(x, LTTextBoxHorizontal)):
                    with open(txt_path, 'a', encoding='utf-8') as f:
                        results = x.get_text()
                        f.write(results + '\n')
 
def solve(name_list, pdf_names):
    excludes = {} # 定义例外的词
    word_list = []
    for name in name_list:
        txt_name = name+'.txt'
        txt_path = os.path.join(root_path, txt_name)
        txt = open(txt_path, "r", encoding='utf-8').read()
        words = jieba.lcut(txt)
        word_list.append(words)

    counts = {}
    for words in word_list:
        for word in words:
            if len(word) == 1:  # 排除单个字符的分词结果
                continue
            else:
                counts[word] = counts.get(word, 0) + 1
        for word in excludes:
            del (counts[word])
    items = list(counts.items())
    items.sort(key=lambda x: x[1], reverse=True)
    for i in range(15):
        word, count = items[i]
        print("{:*<10}{:->5}".format(word, count))
    for name in pdf_names:
        os.remove(name+'.txt')

In [2]:
pdf_name1 = '学校艺术教育工作规程 - 中华人民共和国教育部政府门户网站'
pdf_name2 = '教育部关于切实加强新时代高等学校美育工作的意见 - 中华人民共和国教育部政府门户网站'
pdf_name3 = '教育部关于推进学校艺术教育发展的若干意见 - 中华人民共和国教育部政府门户网站'
txt_name1 = '关于全面加强和改进新时代学校美育工作的意见'
pdf_names = [pdf_name1, pdf_name2, pdf_name3]
txt_names = [txt_name1]
name_list = pdf_names + txt_names
for pdf_name in pdf_names:
    parse_pdf(pdf_name)

In [7]:
solve(name_list, pdf_names)

艺术********-3329
教育********-2174
学校********-1478
美育********-1034
高校********--803
学生********--677
工作********--607
课程********--545
活动********--543
发展********--482
教师********--403
文化********--345
建设********--330
教学********--324
应当********--310
